In [8]:
# Data Analysis for Order Data

# Install required libraries
# pip install kaggle
!pip install kaggle

In [9]:
#import kaggle
import kaggle

# Download the dataset using Kaggle api
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
# Extract the downloaded zip file
import zipfile
zip_ref = zipfile.ZipFile('orders.csv')
zip_ref.extractall('orders')
zip_ref.close()


In [11]:
# Read the dataset using pandas and handle null values
import pandas as pd

df_orders = pd.read_csv('orders/orders.csv', na_values=['Not Available', 'unknown'])

# Display the first few rows of the dataset
df_orders.head()

# Check for unique valus in the 'Ship Mode' column
df_orders['Ship Mode'].unique()



array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [12]:
# Rename the columns for better readability 
df_orders.columns = df_orders.columns.str.lower()
df_orders.columns = df_orders.columns.str.replace(' ', '_')
df_orders.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [18]:
# Define new columns: discount, sale price, and profit  
df_orders['discount'] = df_orders['list_price']*df_orders['discount_percent']/100

df_orders['sale_price'] = df_orders['list_price']-df_orders['discount']

df_orders['profit'] = df_orders['sale_price'] - df_orders['cost_price']

df_orders.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [19]:
# Check the data types of the columns
df_orders.dtypes

order_id              int64
order_date           object
ship_mode            object
segment              object
country              object
city                 object
state                object
postal_code           int64
region               object
category             object
sub_category         object
product_id           object
cost_price            int64
list_price            int64
quantity              int64
discount_percent      int64
discount            float64
sale_price          float64
profit              float64
dtype: object

In [21]:
# Convert the 'order_date' column to datetime format
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'], format="%Y-%m-%d")

# Check the data types again
df_orders.dtypes


order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [22]:
# drop unnecessary columns
df_orders.drop(columns=['cost_price', 'list_price', 'discount_percent'], inplace=True)

In [23]:
df_orders.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0


In [24]:
# Connect to SQL Server using sqlalchemy
import sqlalchemy as sql
engine = sql.create_engine('mssql://LAPTOP-FFGU1JOG/70-461?driver=ODBC+Driver+17+for+SQL+Server')

# Create a connection to the database
conn = engine.connect()

In [25]:
df_orders.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [26]:
# Load dataframe into SQL Server
df_orders.to_sql('orders', con=conn, if_exists='append', index=False)

38